In [1]:
import torch

print("Is CUDA available?", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name())

Is CUDA available? True
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
import torch
import torch.nn as nn

class DoubleConv(nn.Module):
  """( convolution - BatchNormalization - ReLU ) for twice """
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.double_conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
    )
  def forward(self, x):
    return self.double_conv(x)

class Down(nn.Module):
  """ ( MaxPooling - DoubleConvolution ) """
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.maxpool_conv = nn.Sequential(
      nn.MaxPool2d(2),
      DoubleConv(in_channels, out_channels)
    )
  def forward(self, x):
    return self.maxpool_conv(x)

class Up(nn.Module):
  """Upscaling then double conv"""
  def __init__(self, in_channels, out_channels, bilinear=True):
    super().__init__()
    # if bilinear, use the normal convolutions to reduce the number of channels
    if bilinear:
      self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    else:
      self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)

    self.conv = DoubleConv(in_channels, out_channels)
  def forward(self, x1, x2):
    x1 = self.up(x1)
    # input is CHW
    diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
    diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

    x1 = F.pad(x1, [diffX//2, diffX-diffX//2,diffY//2, diffY-diffY//2])
    x = torch.cat([x2, x1], dim=1)
    return self.conv(x)

class OutConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(OutConv, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
  def forward(self, x):
    return self.conv(x)

In [3]:
import torch.nn.functional as F

class UNet(nn.Module):
  def __init__(self, n_channels, n_classes, bilinear=False):
    super(UNet, self).__init__()
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.bilinear = bilinear

    # channel of first layer
    layer = 64

    self.inc = DoubleConv(n_channels, layer)
    self.down1 = Down(layer, layer*2)
    self.down2 = Down(layer*2, layer*4)
    self.down3 = Down(layer*4, layer*8)
    self.down4 = Down(layer*8, layer*16)
    self.up1 = Up(layer*16, layer*8, bilinear)
    self.up2 = Up(layer*8, layer*4, bilinear)
    self.up3 = Up(layer*4, layer*2, bilinear)
    self.up4 = Up(layer*2, layer, bilinear)
    self.outc = OutConv(layer, n_classes)
  def forward(self, x):
    x1 = self.inc(x)
    x2 = self.down1(x1)
    x3 = self.down2(x2)
    x4 = self.down3(x3)
    x5 = self.down4(x4)
    x = self.up1(x5, x4)
    x = self.up2(x, x3)
    x = self.up3(x, x2)
    x = self.up4(x, x1)
    logits = self.outc(x)
    return logits

if __name__ == '__main__':
    net = UNet(n_channels=3, n_classes=1)
    print(net)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [4]:
import torch
import cv2
import os
import glob
from torch.utils.data import Dataset
import random


class ISBI_Loader(Dataset):
  def __init__(self, data_path):
    # 初始化函數，讀取所有data_path下的圖片
    self.data_path = data_path
    self.imgs_path = glob.glob(os.path.join(data_path, 'Flair/*.png'))
  def augment(self, image, flipCode):
    # 使用cv2.flip進行數據增強，filpCode為：1=水平翻轉，0=垂直翻轉，-1=水平+垂直翻轉
    flip = cv2.flip(image, flipCode)
    return flip
  def __getitem__(self, index):
    # 根據index讀取圖片
    image_path = self.imgs_path[index]
    # 根據image_path生成label_path
    label_path = image_path.replace('Flair', 'WMH')
    # 讀取訓練圖片和標籤圖片
    image = cv2.imread(image_path)
    label = cv2.imread(label_path)
    # 將數據轉為單通道的圖片
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
    image = image.reshape(1, image.shape[0], image.shape[1])
    label = label.reshape(1, label.shape[0], label.shape[1])
    # 處理標籤，將像素值为255的改為1
    if label.max() > 0:
        label = label / label.max()
    # 隨機進行數據增強，為2時不做處理
    flipCode = random.choice([2])
    if flipCode != 2:
        image = self.augment(image, flipCode)
        label = self.augment(label, flipCode)
    return image, label
  def __len__(self):
    # 返回训练集大小
    return len(self.imgs_path)

if __name__ == "__main__":
  isbi_dataset = ISBI_Loader("Train")
  print("數據個數：", len(isbi_dataset))
  train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset, batch_size=2, shuffle=True)
  for image, label in train_loader:
    print(image.shape)

數據個數： 21440
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
torch.Size([2, 1, 256, 256])
to

In [5]:
from torch import optim
import torch.nn as nn
import torch

def train_net(net, device, data_path, epochs=100, batch_size=10, lr=0.001, resume=True, model_path='best_model.pth'):
    if resume and os.path.isfile(model_path):
        print(f"Loading model from {model_path}")
        net.load_state_dict(torch.load(model_path, map_location=device))  
    # 加載訓練集
    isbi_dataset = ISBI_Loader(data_path)
    train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset, batch_size=batch_size, shuffle=True)
    # 定義Optimizer算法
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-8)
    # 定義Loss算法
    criterion_bce = nn.BCEWithLogitsLoss()

    # best_loss統計，初始化為正無窮
    best_loss = float('inf')
    # 訓練epochs次
    for epoch in range(epochs):
        # 训练模式
        net.train()
        # 按照batch_size開始訓練
        for Flair, WMH in train_loader:
            optimizer.zero_grad()
            # 將數據複製到device中
            Flair = Flair.to(device=device, dtype=torch.float32)
            WMH = WMH.to(device=device, dtype=torch.float32)
            # 使用神經網路參數，輸出預測結果
            pred = net(Flair)
            # 計算loss
            loss= criterion_bce(pred, WMH)
            print('Loss/train', loss.item(),'epoch',epoch+1)
            # 保存loss值最小的神經網路參數
            if loss < best_loss:
                best_loss = loss
                torch.save(net.state_dict(), 'best_model.pth')
            # 更新参数
            loss.backward()
            optimizer.step()

if __name__ == "__main__":
    # 選擇設備，有cuda用cuda，沒有就用cpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # 輸入神經網路架構。圖片通道1，分類为1。
    net = UNet(n_channels=1, n_classes=1)
    # 將神經網路複製到到deivce中
    net.to(device=device)
    # 指定訓練集位置，開始訓練
    data_path = "Train"
    train_net(net, device, data_path, resume=True)

Loading model from best_model80_epoch105.pth
Loss/train 0.0004911585710942745 epoch 1
Loss/train 0.0013408096274361014 epoch 1
Loss/train 0.0004307015333324671 epoch 1
Loss/train 0.0006727527943439782 epoch 1
Loss/train 0.0006025214097462595 epoch 1
Loss/train 0.00013797437713947147 epoch 1
Loss/train 0.0005477149970829487 epoch 1
Loss/train 0.0005144478636793792 epoch 1
Loss/train 0.00010004189243772998 epoch 1
Loss/train 0.0007659198017790914 epoch 1
Loss/train 0.0011902530677616596 epoch 1
Loss/train 0.00019557299674488604 epoch 1
Loss/train 0.0011297597084194422 epoch 1
Loss/train 0.0008048269082792103 epoch 1
Loss/train 0.0009266819688491523 epoch 1
Loss/train 0.0008656801655888557 epoch 1
Loss/train 0.0005437593790702522 epoch 1
Loss/train 0.00028261455008760095 epoch 1
Loss/train 0.0010721925646066666 epoch 1
Loss/train 0.0006200945936143398 epoch 1
Loss/train 0.0005556810065172613 epoch 1
Loss/train 0.0006660603103227913 epoch 1
Loss/train 0.0005060436087660491 epoch 1
Loss/tra

In [4]:
import glob
import numpy as np
import torch
import os
import cv2

if __name__ == "__main__":
    # 選擇設備，有cuda用cuda，沒有就用cpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # 加載神經網路，圖片單通道，分類為1。
    net = UNet(n_channels=1, n_classes=1)
    # 將網路複製到deivce中
    net.to(device=device)
    # 加載模型參數
    net.load_state_dict(torch.load('best_model.pth', map_location=device))
    # 測試模式
    net.eval()
    # 讀取圖片路徑
    tests_path = glob.glob('image_folder/*.png')
    # 讀取所有圖片
    for test_path in tests_path:
        # 保存结果
        save_res_path = test_path.split('.')[0] + '_res.png'
        # 讀取圖片
        img = cv2.imread(test_path)
        # 轉為灰階圖片
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # 轉為batch為1，通道為1，大小為512*512的數據
        img = img.reshape(1, 1, img.shape[0], img.shape[1])
        # 轉為tensor
        img_tensor = torch.from_numpy(img)
        # 將tensor複製到device中，只用cpu就是複製到cpu中，用cuda就是複製到cuda中。
        img_tensor = img_tensor.to(device=device, dtype=torch.float32)
        # 預測
        pred = net(img_tensor)
        # 提取结果
        pred = np.array(pred.data.cpu()[0])[0]

        print(np.max(pred))
        # 處理结果
        pred[pred >= 0] = 255
        pred[pred < 0] = 0
        # 保存圖片
        cv2.imwrite(save_res_path, pred)

-6.578247
-8.022676
-8.371211
-4.4439826
6.719423
3.9217663
-0.86095107
-2.2816703
-1.131009
-4.709301
-5.882914
0.38560593
-0.3558185
-1.4478734
0.5990605
-0.8933315
-4.71654
-3.0305588
-4.6065955
-3.332641
-8.287202
-3.531931
-6.68972
-7.9203935
-8.306184
-4.353564
-1.8403901
-2.1880345
0.97728014
-1.6594787
-5.425342
-6.5238404
-5.6008935
-5.887755
-2.9312863
-3.750711
-9.202736
-6.5778985
-7.3040433
-6.3279576
-6.0790734
-2.276023
0.027488455
3.7801385
3.3828707
7.26821
-1.7693653
-2.8902175
-3.0556748
-0.4726261
6.3510346
6.8774467
11.845506
0.8400566
2.621813
3.9096904
3.947596
3.3464518
5.9354897
4.4517593
-2.146381
-2.7066555
-2.411306
4.6032953
-2.4734206
14.172793
13.4126425
10.367448
5.980615
5.3892593
6.5250106
0.8266872
5.1431212
1.602223
7.206202
4.918834
4.345794
-1.8488467
2.5689588
1.6456375
7.5896153
1.4956176
2.5446172
9.241054
11.376536
13.417083
11.07758
7.8579144
5.866219
5.9687514
7.0192094
2.5115628
3.4854646
10.037776
9.693301
7.219834
8.793213
5.621082
10.4506